Вариант 1: полностью напишите SQL скрипт, который автоматически посчитает дополнительные продажи.

Дано: 3 маркетинговые компании. В каждой из компаний есть целевая(была коммуникация) и контрольная группы. В целевой группе необходимо учитывать только тех клиентов, кому коммуникация была доставлена. 
код доставки сообщения:

1- сообщение доставлено

2 - сообщение потеряно

3 - клиент находится в контрольной группе (отправка заблокирована системой)
следовательно первым делом необходимо удалить из таблицы response пользователей, которые не находятся в контрольной группе, но до которых не дошла коммуникация(потерялось сообщение). Будет лучше создать представление. Для каждой кампании создадим свое представление:

Для маркетинговой кампании А_Б тест макета банера в письме (А)

In [ ]:
create or replace
view AB_responses_A
as select * from project.response where cell_package_sk = 118512 and response_code = 1

create or replace
view AB_responses_A_ctr
as select * from project.response where cell_package_sk = 118513 


Б

In [ ]:
create or replace
view AB_responses_B
as select * from project.response where cell_package_sk = 118514 and response_code = 1

create or replace
view AB_responses_B_ctr
as select * from project.response where cell_package_sk = 118515

Скидочная акция 

In [ ]:
create or replace
view sales_response
as select * from project.response where cell_package_sk = 118516 and response_code = 1

create or replace
view sales_response_ctr
as select * from project.response where cell_package_sk = 118517

Специальное предложение

In [ ]:
create or replace
view offer
as select * from project.response where cell_package_sk = 118518 and response_code = 1

create or replace
view offer_ctr
as select * from project.response where cell_package_sk = 118519

Теперь последовательно для каждой кампании составим по две таблицы с покупками, для целевой и контрольной группы. Необходимо учитывать то, что покупки могли быть совершены до\после заврешения маркетинговой кампании, поэтому возьмем только те покупки, которые совершены в требуемый диапазон времени

Кампания по АБ тестированию баннера началась 2023-06-23 и закончилась через три дня.

In [ ]:
create or replace 
view A_sales
as select AB_responses_A."Id_customer",  sales."Date", sales."id_product" from AB_responses_A, sales where AB_responses_A."Id_customer" = sales."Id_customer"
and sales."Date" in ('2023-06-23 00:00:00', '2023-06-24 00:00:00', '2023-06-25 00:00:00')

create or replace 
view A_sales_ctr
as select AB_responses_A_ctr."Id_customer",  sales."Date", sales."id_product" from AB_responses_A_ctr, sales where AB_responses_A_ctr."Id_customer" = sales."Id_customer"
and sales."Date" in ('2023-06-23 00:00:00', '2023-06-24 00:00:00', '2023-06-25 00:00:00')

для Б

In [ ]:
create or replace 
view B_sales
as select AB_responses_B."Id_customer",  sales."Date", sales."id_product" from AB_responses_B, sales where AB_responses_B."Id_customer" = sales."Id_customer"
and sales."Date" in ('2023-06-23 00:00:00', '2023-06-24 00:00:00', '2023-06-25 00:00:00')

create or replace 
view B_sales_ctr
as select AB_responses_B_ctr."Id_customer",  sales."Date", sales."id_product" from AB_responses_B_ctr, sales where AB_responses_B_ctr."Id_customer" = sales."Id_customer"
and sales."Date" in ('2023-06-23 00:00:00', '2023-06-24 00:00:00', '2023-06-25 00:00:00')

скидочная акция началась 2023-06-27 + 3 дня 

In [ ]:
create or replace 
view sales_response_result
as select sales_response."Id_customer",  sales."Date", sales."id_product" from sales_response, sales where sales_response."Id_customer" = sales."Id_customer"
and sales."Date" in ('2023-06-27 00:00:00', '2023-06-28 00:00:00', '2023-06-29 00:00:00')


create or replace 
view sales_response_ctr_result
as select sales_response_ctr."Id_customer",  sales."Date", sales."id_product" from sales_response_ctr, sales where sales_response_ctr."Id_customer" = sales."Id_customer"
and sales."Date" in ('2023-06-27 00:00:00', '2023-06-28 00:00:00', '2023-06-29 00:00:00')

Спецпредложение 2023-07-03 + 3 дня

In [ ]:
create or replace 
view offer_sales
as select offer."Id_customer",  sales."Date", sales."id_product" from offer, sales where offer."Id_customer" = sales."Id_customer"
and sales."Date" in ('2023-07-03 00:00:00', '2023-07-04 00:00:00', '2023-07-05 00:00:00')

create or replace 
view offer_sales_ctr
as select offer_ctr."Id_customer",  sales."Date", sales."id_product" from offer_ctr, sales where offer_ctr."Id_customer" = sales."Id_customer"
and sales."Date" in ('2023-07-03 00:00:00', '2023-07-04 00:00:00', '2023-07-05 00:00:00')

сделаем еще 2 приготовления для каждой кампании - создадим вью со сводными таблицами цен для того, чтобы в будущем посчитать сумму покупок в одно действие

In [ ]:
create or replace 
view A_sales_prices as
select a_sales."id_product", price."price"
from a_sales
join price
on price."product_id" = a_sales."id_product"

create or replace 
view A_sales_prices_ctr as
select a_sales_ctr."id_product", price."price"
from a_sales_ctr
join price
on price."product_id" = a_sales_ctr."id_product"

select * from A_sales_prices_ctr

и заранее подсчитаем конверсии

In [ ]:
 create or replace view a_convert as select ROUND(
        COUNT(DISTINCT a_sales."Id_customer")::DECIMAL / 
        COUNT(DISTINCT ab_responses_a."Id_customer"),
    4) as conversion_ratio from a_sales, ab_responses_a
    
    
    
create or replace view a_ctr_convert as select ROUND(
        COUNT(DISTINCT a_sales_Ctr."Id_customer")::DECIMAL / 
        COUNT(DISTINCT ab_responses_a_ctr."Id_customer"),
    4) as conversion_ratio from a_sales_ctr, ab_responses_a_ctr



конверсии для Б ветки теста 

In [ ]:
 create or replace view b_convert as select ROUND(
        COUNT(DISTINCT b_sales."Id_customer")::DECIMAL / 
        COUNT(DISTINCT ab_responses_b."Id_customer"),
    4) as conversion_ratio from b_sales, ab_responses_b

    
create or replace view b_ctr_convert as select ROUND(
        COUNT(DISTINCT b_sales_Ctr."Id_customer")::DECIMAL / 
        COUNT(DISTINCT ab_responses_b_ctr."Id_customer"),
    4) as conversion_ratio from b_sales_ctr, ab_responses_b_ctr

create or replace 
view b_sales_prices as
select b_sales."id_product", price."price"
from b_sales
join price
on price."product_id" = b_sales."id_product"

create or replace 
view b_sales_prices_ctr as
select b_sales_ctr."id_product", price."price"
from b_sales_ctr
join price
on price."product_id" = b_sales_ctr."id_product"

select * from b_sales_prices_ctr

конверсии для оффера и таблицы цен 

In [ ]:
 create or replace view offer_convert as select ROUND(
        COUNT(DISTINCT offer_sales."Id_customer")::DECIMAL / 
        COUNT(DISTINCT offer."Id_customer"),
    4) as conversion_ratio from offer_sales, offer

select * from offer_convert

create or replace view offer_ctr_convert as select ROUND(
        COUNT(DISTINCT offer_sales_ctr."Id_customer")::DECIMAL / 
        COUNT(DISTINCT offer_ctr."Id_customer"),
    4) as conversion_ratio from offer_sales_ctr, offer_ctr
    
select * from offer_ctr_convert

create or replace 
view offer_sales_prices as
select offer_sales."id_product", price."price"
from offer_sales
join price
on price."product_id" = offer_sales."id_product"

create or replace 
view offer_sales_prices_ctr as
select offer_sales_ctr."id_product", price."price"
from offer_sales_ctr
join price
on price."product_id" = offer_sales_ctr."id_product"

select * from offer_sales_prices_ctr

конверсии для скидочной акции и таблицы цен

In [ ]:
CREATE OR REPLACE VIEW sales_convert AS 
SELECT ROUND(
    (SELECT COUNT(DISTINCT "Id_customer") FROM sales_response_result)::DECIMAL /
    (SELECT COUNT(DISTINCT "Id_customer") FROM sales_response),
    4
) as conversion_ratio;

select * from sales_convert

create or replace view sales_ctr_convert as select ROUND(
        COUNT(DISTINCT sales_response_ctr_result."Id_customer")::DECIMAL / 
        COUNT(DISTINCT sales_response_ctr."Id_customer"),
    4) as conversion_ratio from sales_response_ctr_result, sales_response_ctr
    
select * from sales_ctr_convert


create or replace 
view sales_prices as
select sales_response_result."id_product", price."price"
from sales_response_result
join price
on price."product_id" = sales_response_result."id_product"

select * from sales_prices

create or replace 
view sales_prices_ctr as
select sales_response_ctr_result."id_product", price."price"
from sales_response_ctr_result
join price
on price."product_id" = sales_response_ctr_result."id_product"

select * from sales_prices_ctr


Теперь создадим финальную сводную таблицу с дополнительной выручкой по каждой из маркетинговых компаний и попробуем сделать выводы

In [ ]:
create table additional_revenue
(ab_a numeric,
ab_b numeric,
sales numeric,
offer numeric)

insert into additional_revenue (ab_a, ab_b, sales, offer)
values ((SELECT 
    (((SELECT a_convert."conversion_ratio" FROM a_convert) - 
     (SELECT a_ctr_convert."conversion_ratio" FROM a_ctr_convert)) * 
     COUNT(ab_responses_a."Id_customer")) * (round((SELECT SUM("price") FROM a_sales_prices)::DECIMAL /
    (SELECT COUNT(DISTINCT "Id_customer") FROM a_sales), 0))
FROM ab_responses_a), 
(SELECT 
    (((SELECT b_convert."conversion_ratio" FROM b_convert) - 
     (SELECT b_ctr_convert."conversion_ratio" FROM b_ctr_convert)) * 
     COUNT(ab_responses_b."Id_customer")) * (round((SELECT SUM("price") FROM b_sales_prices)::DECIMAL /
    (SELECT COUNT(DISTINCT "Id_customer") FROM b_sales), 0))
FROM ab_responses_b),
(SELECT 
    (((select sales_convert."conversion_ratio" FROM sales_convert) - 
     (SELECT sales_ctr_convert."conversion_ratio" FROM sales_ctr_convert)) * 
     COUNT(sales_response."Id_customer")) * (round((SELECT SUM("price") FROM sales_prices)::DECIMAL /
    (SELECT COUNT(DISTINCT "Id_customer") FROM sales_response_result), 0))
FROM sales_response),

(SELECT 
    (((select offer_convert."conversion_ratio" FROM offer_convert) - 
     (SELECT offer_ctr_convert."conversion_ratio" FROM offer_ctr_convert)) * 
     COUNT(offer."Id_customer")) * (round((SELECT SUM("price") FROM offer_sales_prices)::DECIMAL /
    (SELECT COUNT(DISTINCT "Id_customer") FROM offer_sales), 0))
FROM offer))

Выгрузим получившуюся таблицу в csv и импортируем в питон

In [3]:
import pandas as pd
data = pd.read_csv('additional_revenue.csv')
data

,ab_a,ab_b,sales,offer
0,3654.1125,-500.7016,88374.4205,91212.8778


маркетинговая компания, связанная с АБ тестированием вариантов баннера показала не впечатляющие результаты, особенно Б ветка теста - дополнительного дохода извлечь не удалось, наоборот, контрольная группа, которой показывали изначальную, старую версию баннера, потратила больше денег.
Скидочная кампания и спецпредложение показали себя намного лучше, что видно из таблицы (disclaimer выводы актуальны только в случае, если все операции выше были правильными)

полный листинг sql скрипта:

In [ ]:
create or replace
view AB_responses_A
as select * from project.response where cell_package_sk = 118512 and response_code = 1

create or replace
view AB_responses_A_ctr
as select * from project.response where cell_package_sk = 118513 


create or replace
view AB_responses_B
as select * from project.response where cell_package_sk = 118514 and response_code = 1

create or replace
view AB_responses_B_ctr
as select * from project.response where cell_package_sk = 118515


create or replace
view sales_response
as select * from project.response where cell_package_sk = 118516 and response_code = 1

create or replace
view sales_response_ctr
as select * from project.response where cell_package_sk = 118517


create or replace 
view A_sales
as select AB_responses_A."Id_customer",  sales."Date", sales."id_product" from AB_responses_A, sales where AB_responses_A."Id_customer" = sales."Id_customer"
and sales."Date" in ('2023-06-23 00:00:00', '2023-06-24 00:00:00', '2023-06-25 00:00:00')

create or replace 
view A_sales_ctr
as select AB_responses_A_ctr."Id_customer",  sales."Date", sales."id_product" from AB_responses_A_ctr, sales where AB_responses_A_ctr."Id_customer" = sales."Id_customer"
and sales."Date" in ('2023-06-23 00:00:00', '2023-06-24 00:00:00', '2023-06-25 00:00:00')


create or replace 
view B_sales
as select AB_responses_B."Id_customer",  sales."Date", sales."id_product" from AB_responses_B, sales where AB_responses_B."Id_customer" = sales."Id_customer"
and sales."Date" in ('2023-06-23 00:00:00', '2023-06-24 00:00:00', '2023-06-25 00:00:00')

create or replace 
view B_sales_ctr
as select AB_responses_B_ctr."Id_customer",  sales."Date", sales."id_product" from AB_responses_B_ctr, sales where AB_responses_B_ctr."Id_customer" = sales."Id_customer"
and sales."Date" in ('2023-06-23 00:00:00', '2023-06-24 00:00:00', '2023-06-25 00:00:00')


create or replace 
view sales_response_result
as select sales_response."Id_customer",  sales."Date", sales."id_product" from sales_response, sales where sales_response."Id_customer" = sales."Id_customer"
and sales."Date" in ('2023-06-27 00:00:00', '2023-06-28 00:00:00', '2023-06-29 00:00:00')

create or replace 
view sales_response_ctr_result
as select sales_response_ctr."Id_customer",  sales."Date", sales."id_product" from sales_response_ctr, sales where sales_response_ctr."Id_customer" = sales."Id_customer"
and sales."Date" in ('2023-06-27 00:00:00', '2023-06-28 00:00:00', '2023-06-29 00:00:00')


create or replace 
view offer_sales
as select offer."Id_customer",  sales."Date", sales."id_product" from offer, sales where offer."Id_customer" = sales."Id_customer"
and sales."Date" in ('2023-07-03 00:00:00', '2023-07-04 00:00:00', '2023-07-05 00:00:00')

create or replace 
view offer_sales_ctr
as select offer_ctr."Id_customer",  sales."Date", sales."id_product" from offer_ctr, sales where offer_ctr."Id_customer" = sales."Id_customer"
and sales."Date" in ('2023-07-03 00:00:00', '2023-07-04 00:00:00', '2023-07-05 00:00:00')


create or replace 
view A_sales_prices as
select a_sales."id_product", price."price"
from a_sales
join price
on price."product_id" = a_sales."id_product"

create or replace 
view A_sales_prices_ctr as
select a_sales_ctr."id_product", price."price"
from a_sales_ctr
join price
on price."product_id" = a_sales_ctr."id_product"

create or replace view a_convert as select ROUND(
        COUNT(DISTINCT a_sales."Id_customer")::DECIMAL / 
        COUNT(DISTINCT ab_responses_a."Id_customer"),
    4) as conversion_ratio from a_sales, ab_responses_a    
    
create or replace view a_ctr_convert as select ROUND(
        COUNT(DISTINCT a_sales_Ctr."Id_customer")::DECIMAL / 
        COUNT(DISTINCT ab_responses_a_ctr."Id_customer"),
    4) as conversion_ratio from a_sales_ctr, ab_responses_a_ctr


 create or replace view b_convert as select ROUND(
        COUNT(DISTINCT b_sales."Id_customer")::DECIMAL / 
        COUNT(DISTINCT ab_responses_b."Id_customer"),
    4) as conversion_ratio from b_sales, ab_responses_b

    
create or replace view b_ctr_convert as select ROUND(
        COUNT(DISTINCT b_sales_Ctr."Id_customer")::DECIMAL / 
        COUNT(DISTINCT ab_responses_b_ctr."Id_customer"),
    4) as conversion_ratio from b_sales_ctr, ab_responses_b_ctr

create or replace 
view b_sales_prices as
select b_sales."id_product", price."price"
from b_sales
join price
on price."product_id" = b_sales."id_product"

create or replace 
view b_sales_prices_ctr as
select b_sales_ctr."id_product", price."price"
from b_sales_ctr
join price
on price."product_id" = b_sales_ctr."id_product"


create or replace view offer_convert as select ROUND(
        COUNT(DISTINCT offer_sales."Id_customer")::DECIMAL / 
        COUNT(DISTINCT offer."Id_customer"),
    4) as conversion_ratio from offer_sales, offer

create or replace view offer_ctr_convert as select ROUND(
        COUNT(DISTINCT offer_sales_ctr."Id_customer")::DECIMAL / 
        COUNT(DISTINCT offer_ctr."Id_customer"),
    4) as conversion_ratio from offer_sales_ctr, offer_ctr

create or replace 
view offer_sales_prices as
select offer_sales."id_product", price."price"
from offer_sales
join price
on price."product_id" = offer_sales."id_product"

create or replace 
view offer_sales_prices_ctr as
select offer_sales_ctr."id_product", price."price"
from offer_sales_ctr
join price
on price."product_id" = offer_sales_ctr."id_product"


CREATE OR REPLACE VIEW sales_convert AS 
SELECT ROUND(
    (SELECT COUNT(DISTINCT "Id_customer") FROM sales_response_result)::DECIMAL /
    (SELECT COUNT(DISTINCT "Id_customer") FROM sales_response),
    4
) as conversion_ratio;

create or replace view sales_ctr_convert as select ROUND(
        COUNT(DISTINCT sales_response_ctr_result."Id_customer")::DECIMAL / 
        COUNT(DISTINCT sales_response_ctr."Id_customer"),
    4) as conversion_ratio from sales_response_ctr_result, sales_response_ctr


create or replace 
view sales_prices as
select sales_response_result."id_product", price."price"
from sales_response_result
join price
on price."product_id" = sales_response_result."id_product"

create or replace 
view sales_prices_ctr as
select sales_response_ctr_result."id_product", price."price"
from sales_response_ctr_result
join price
on price."product_id" = sales_response_ctr_result."id_product"

create table additional_revenue
(ab_a numeric,
ab_b numeric,
sales numeric,
offer numeric)

insert into additional_revenue (ab_a, ab_b, sales, offer)
values ((SELECT 
    (((SELECT a_convert."conversion_ratio" FROM a_convert) - 
     (SELECT a_ctr_convert."conversion_ratio" FROM a_ctr_convert)) * 
     COUNT(ab_responses_a."Id_customer")) * (round((SELECT SUM("price") FROM a_sales_prices)::DECIMAL /
    (SELECT COUNT(DISTINCT "Id_customer") FROM a_sales), 0))
FROM ab_responses_a), 
(SELECT 
    (((SELECT b_convert."conversion_ratio" FROM b_convert) - 
     (SELECT b_ctr_convert."conversion_ratio" FROM b_ctr_convert)) * 
     COUNT(ab_responses_b."Id_customer")) * (round((SELECT SUM("price") FROM b_sales_prices)::DECIMAL /
    (SELECT COUNT(DISTINCT "Id_customer") FROM b_sales), 0))
FROM ab_responses_b),
(SELECT 
    (((select sales_convert."conversion_ratio" FROM sales_convert) - 
     (SELECT sales_ctr_convert."conversion_ratio" FROM sales_ctr_convert)) * 
     COUNT(sales_response."Id_customer")) * (round((SELECT SUM("price") FROM sales_prices)::DECIMAL /
    (SELECT COUNT(DISTINCT "Id_customer") FROM sales_response_result), 0))
FROM sales_response),

(SELECT 
    (((select offer_convert."conversion_ratio" FROM offer_convert) - 
     (SELECT offer_ctr_convert."conversion_ratio" FROM offer_ctr_convert)) * 
     COUNT(offer."Id_customer")) * (round((SELECT SUM("price") FROM offer_sales_prices)::DECIMAL /
    (SELECT COUNT(DISTINCT "Id_customer") FROM offer_sales), 0))
FROM offer))
